In [26]:
import numpy as np
import plotly.graph_objects as go
import time
import ipywidgets as widgets
from IPython.display import display

# Алгоритм смешивания параметров
size = 20
p1 = 0.45
p2 = 0.45
num_for_happy = 2
max_num_iter = 100

# Генерация начального мира
color_map_flat = np.zeros((size**2))
elements = size ** 2
num_1 = round(elements * p1)
num_2 = round(elements * p2)
color_map_flat[0:num_1] = -0.6  # отрицательное значение
color_map_flat[num_1: num_2+num_1] = 0.6  # положительное значение
np.random.shuffle(color_map_flat)
color_map = color_map_flat.reshape((size, size))

# Функция для вычисления счастья клетки
def cell_happiness(world, i, j, num_same_for_happiness):
    cur_happiness = 0
    cur_cell_value = world[i, j]
    x, y = world.shape
    if j + 1 < y:
        if cur_cell_value == world[i, j + 1]:
            cur_happiness += 1

    if j - 1 >= 0:
        if cur_cell_value == world[i, j - 1]:
            cur_happiness += 1

    if (i + 1 < x) & (j - 1 >= 0):
        if cur_cell_value == world[i + 1, j - 1]:
            cur_happiness += 1

    if i + 1 < x:
        if cur_cell_value == world[i + 1, j]:
            cur_happiness += 1

    if (i + 1 < x) & (j + 1 < y):
        if cur_cell_value == world[i + 1, j + 1]:
            cur_happiness += 1

    if (i - 1 >= 0) & (j - 1 >= 0):
        if cur_cell_value == world[i - 1, j - 1]:
            cur_happiness += 1

    if (i - 1 >= 0) & (j + 1 < y):
        if cur_cell_value == world[i - 1, j + 1]:
            cur_happiness += 1

    if i - 1 >= 0:
        if cur_cell_value == world[i - 1, j]:
            cur_happiness += 1

    if cur_happiness >= num_same_for_happiness:
        return True
    return False

# Функция для обновления визуализации
def update_figure(iteration):
    fig.data[0].z = color_map
    fig.update_layout(title_text=f"Итерация {iteration + 1}")
    fig.show()
    time.sleep(0.5)

    # Итеративное перемещение случайной несчастной клетки в случайную пустую клетку
fig = go.Figure(data=go.Heatmap(z=color_map, colorscale='piyg'))
fig.update_layout(title_text="Итерация 0")
fig.show()

# Создаем кнопку для обновления
button = widgets.Button(description="Следующая итерация")
display(button)

def on_button_click(b):
    for iteration in range(max_num_iter):
        # Вычисление карты счастья
        happiness_map = np.zeros_like(color_map)
        for i, _ in enumerate(happiness_map):
            for j, _ in enumerate(happiness_map[i]):
                happiness_map[i, j] = cell_happiness(color_map, i, j, num_for_happy)
        unhappy_x, unhappy_y = np.where((happiness_map == 0) & (color_map != 0))
        empty_x, empty_y = np.where(color_map == 0)

        if len(unhappy_x) != 0:
            unhappy_to_move_idx = np.random.randint(0, len(unhappy_x))
            empty_to_move_idx = np.random.randint(0, len(empty_x))
            color_map[empty_x[empty_to_move_idx], empty_y[empty_to_move_idx]] =\
                color_map[unhappy_x[unhappy_to_move_idx ], unhappy_y[unhappy_to_move_idx]]
            color_map[unhappy_x[unhappy_to_move_idx ], unhappy_y[unhappy_to_move_idx]] = 0

            update_figure(iteration)
        else:
            print("После итерации", iteration, "модель завершила работу, несчастливых клеток не осталось")
            break

button.on_click(on_button_click)

Button(description='Следующая итерация', style=ButtonStyle())

После итерации 65 модель завершила работу, несчастливых клеток не осталось
